## 3.4.2 Data Preparation for Integrating Sources

This section teaches how to integrate multiple data sources to obtain more dimensions for analysis and modeling. To practically learn how to implement the methods, we return to the mini-case study of customer churn. We have observed three available datasets. The following summarizes the datasets, their source file name, and the unique key (i.e., the attribute that its value can uniquely identify a row of a dataset)

- Customer (customer.xlsx): the unique key is the CustomerId attribute.
- Churn (churn.xlsx): the unique key is the CustomerId attribute.
- Payment transaction (ptransaction.xlsx): the unique key is the CustomerID attribute.

Based on the unique key of each dataset of Customer, Churn, and Payment transaction, we can join them together using the CustomerId attribute.

### Identifying and Removing Duplication
Before joining the datasets, it is essential to check if duplications exist to avoid redundancy. The following example of Python codes can be used to find duplications with the duplicated() function.

In [1]:
# import library and access data sources
import pandas as pd
import math
from datetime import datetime, timedelta

df1 = pd.read_excel ('data/customer.xlsx')
df2 = pd.read_excel ('data/churn.xlsx')
df3 = pd.read_excel ('data/ptransaction.xlsx')

# identify if duplication exists 
print(df1[df1.duplicated()])
print(df2[df2.duplicated()])
print(df3[df3.duplicated()])

Empty DataFrame
Columns: [CustomerId, Firstname, Gender, PostalCode, HashCode, Birthdate]
Index: []
   CustomerId
4           6
Empty DataFrame
Columns: [CustomerId, LastTrxDate, MinTrxValue, MaxTrxValue, TotalTrxValue, Cash, Cheque, CreditCard, SinceLastTrx]
Index: []


If there are duplications, then we can use the following example codes to remove the duplications using the drop_duplicates() function to find duplicates and the len() to count the number of duplicates that exist. If none, then proceed to the data types exploration.

In [2]:
# if duplication exists, remove and keep only one 
if len(df1[df1.duplicated()]) > 0: 
    df1 = df1.drop_duplicates(keep = 'first') 
if len(df2[df2.duplicated()]) > 0: 
    df2 = df2.drop_duplicates(keep = 'first') 
if len(df3[df3.duplicated()]) > 0: 
    df3 = df3.drop_duplicates(keep = 'first') 

### Identifying and Converting Unmatched Data Types 
As we have observed the data type of each attribute in the Customer dataset in the previous section 4.1, four attributes' data types need to be corrected. The Firstname, HashCode, PostalCode and Gender should be set as string type. The following Python commands serve the conversion for correction.

In [3]:
# convert data type 
df1['Firstname'] = df1['Firstname'].astype('string') 
df1['PostalCode'] = df1['PostalCode'].astype('string') 
df1['HashCode'] = df1['HashCode'].astype('string') 
df1['Gender'] = df1['Gender'].astype('string')

### Creating a New Attribute to Indicate the Target 'Churn'
The Churn dataset contains all the `CustomerId` of customers that have already churned but do not have an attribute to indicate the classification target, i.e. Churn or not Churn. For this case study, we use the value of 1 to indicate `churn`, else 0.  

To have an indicator of customer churn, we create a new attribute called Churn (i.e. Dummy attribute) as the indicator with a value equal to 'yes' denotes a churned customer. Next, declare the data type of the new attribute to the character type using the astype('string') function. 

In [4]:
# create a dummy variable, naming it Churn attribute to indicate churn with a value 'yes'
df2['Churn'] = 'yes' 
df2['Churn'] = df2['Churn'].astype('string')

### Creating a New Derived Attribute for Age 
In the `Customer` dataset, we observe the `Birthdate` attribute. However, the attribute carries a hidden meaning of age. Therefore, creating and deriving a new attribute Age, to represent age better reflects the meaning. Because the case study data was collected until the end of February 2022, we calculate a customer's age based on this timeline. The following codes show the formula of age calculation according to the timeline.

In [5]:
# create Age Attribute deriving from Birthdate.
df1['Age'] = (datetime(2022, 3, 1) - df1['Birthdate']) // timedelta(days=365.2425)

### Integrating Data Sources
We can use the Python `merge()` function to integrate all data sources. We perform a left join to merge datasets for the implementation by ensuring all rows from the Customer dataset are captured. We want to perform churn classification modeling based on customers' profiles.

In [6]:
#returns all rows from the Customer, even if there are no matches in Churn dataset
df_merge = df1.merge(df2, on='CustomerId', how='left')

#returns all rows from the Customer, even if there are no matches in Transaction
df_merge = df_merge.merge(df3, on='CustomerId', how='left')

### Further Pre-processing to ensure Data Quality
To further prepare the merged dataset, there several pre-processing tasks are necessary. Observe the missing values of each attribute after the merge using the `info()` function. 

In [7]:
# observe the dataset attributes and data types
print(df_merge.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CustomerId     1000 non-null   int64         
 1   Firstname      1000 non-null   string        
 2   Gender         999 non-null    string        
 3   PostalCode     1000 non-null   string        
 4   HashCode       1000 non-null   string        
 5   Birthdate      999 non-null    datetime64[ns]
 6   Age            999 non-null    float64       
 7   Churn          503 non-null    string        
 8   LastTrxDate    998 non-null    datetime64[ns]
 9   MinTrxValue    998 non-null    float64       
 10  MaxTrxValue    998 non-null    float64       
 11  TotalTrxValue  998 non-null    float64       
 12  Cash           998 non-null    float64       
 13  Cheque         998 non-null    float64       
 14  CreditCard     998 non-null    float64       
 15  SinceLastTrx   998 non

Based on the `info()` result, several attributes have missing values. Next, we examine how these missing values can be treated according to the meaning of each attribute:

- When we created a dummy variable, i.e., the Churn attribute in the Churn dataset, we only captured those customers who have already churned. During merging between Customer and Churn datasets, only customers who have churned contain a value of 1. Those not churned have a null value. We will replace the null or missing value with 0, indicating the non-churn status.
- The payment transaction dataset does not contain all customers' payment details, i.e., some customers do not have payment transactions. Also, we observe that one Customer dataset row does not contain gender information. Consequently, null or missing values will exist in the merged dataset. To tackle this problem, we can either delete those rows having missing values or replace the null data with a value. Based on the problem understanding, the classification requires understanding customers' demographics and payment transactions' impact on churn. Thus, we will delete the rows that have missing transaction data.

### Identifying and Treating Missing Values
Besides the `info()` function, we can use another option to explore the existence of the missing values in a specific attribute using the `isna()` function. The example codes are as follows. 

In [8]:
for i, col in enumerate(df_merge.columns): 
    print(col, df_merge[col].isna().sum())

CustomerId 0
Firstname 0
Gender 1
PostalCode 0
HashCode 0
Birthdate 1
Age 1
Churn 497
LastTrxDate 2
MinTrxValue 2
MaxTrxValue 2
TotalTrxValue 2
Cash 2
Cheque 2
CreditCard 2
SinceLastTrx 2


Comparing the `info()` and `isna()` functions, `info()` gives the number of not null values and information on the data type of each attribute in a dataset, whereas `isna()` simply shows the number of missing values that exist in each attribute.

We perform the following tasks based on the rationale for pre-processing using the fillna() function to identify missing values/null and fill in a value into a specific attribute's data. 

We use the `apply(lambda x : math.floor(x)` function to set the data type of the attributes to an integer, and use a 'no' value for the Churn attribute to replace the missing values for those customers who have not churned. 

In [9]:
# to treat null or missing values 
df_merge['Churn'] = df_merge['Churn'].fillna('no') 
df_merge['Cash'] = df_merge['Cash'].fillna(0)
df_merge['Cash'] = df_merge['Cash'].apply(lambda x : math.floor(x))
df_merge['CreditCard'] = df_merge['CreditCard'].fillna(0)
df_merge['CreditCard'] = df_merge['CreditCard'].apply(lambda x : math.floor(x))
df_merge['Cheque'] = df_merge['Cheque'].fillna(0)
df_merge['Cheque'] = df_merge['Cheque'].apply(lambda x : math.floor(x))

We have now treated missing values with a value of 0 to reflect their meaning in the respective attributes. For the rest of the attributes containing missing values, such as `Gender`, `LastTrxDate`, and `SinceLastTrx` (derived from the `LastTrxDate`), we delete the rows because replacing them with any value in the data is meaningless.

We use the `dropna()` function to drop all the rows with at least one missing in any attribute.

In [10]:
# to delete the remaining rows with missing values in any attribute
df_merge.dropna(inplace=True)

### Performing Data Transformation using Replacement
In a real-world scenario, a postal code represents a specific area. For example, we may use the `PostalCode` to identify a customer's living area. However, directly taking the original postal code with many levels (i.e., different values) in the `Customer` dataset may cause the classification model's ability to generalize the future data for prediction. This problem is called overfitting. 

`We will look at the overfitting problem in mode detail in Week 6. `

To avoid overfitting problems due to too many levels in the PostalCode attribute, we use the first character of the postal code to analyze a broader area instead of discarding the information totally. To realize this deployment, we replace the original postal code with the first character of the value.

To select the first character of the attribute, we use the following Python code to implement the selection and set it as a string type.

In [11]:
# to select the first character of the PostalCode
df_merge['PostalCode'] = df_merge['PostalCode'].str[0].astype('string')

### Selecting Relevant Attributes

The previous steps have integrated all data sources and prepared the attributes to reflect the correct meaning and data types. However, some irrelevant such as `HashCode`, represent a meaningless computer-generated value. Moreover, a duplicated attribute such as `Birthdate` that the `Age` has better reflects the meaning. Besides, the `LastTrxDate` values have been derived and reflected in the `SinceLastTrx` attribute. Thus, we will exclude these three attributes and select the rest of the other attributes.

We use the following codes to implement the selection.

In [12]:
# to select useful attributes for further analysis and modeling
df_merge = df_merge[['CustomerId','Gender', 'Age','PostalCode', 'MinTrxValue', 
                'MaxTrxValue', 'TotalTrxValue', 'Cash', 'CreditCard', 'Cheque', 
                'SinceLastTrx', 'Churn']]

### Saving the Consolidated Data
We have now completed the data preparation tasks for the case study to bring the consolidated data for further analysis. Observe the final dataset dimension, attributes, and data types to confirm their structure for further analysis.

In [13]:
# observe the consolidated data dimension and data types
print(df_merge.shape)
print(df_merge.info())

(996, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 996 entries, 0 to 999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CustomerId     996 non-null    int64  
 1   Gender         996 non-null    string 
 2   Age            996 non-null    float64
 3   PostalCode     996 non-null    string 
 4   MinTrxValue    996 non-null    float64
 5   MaxTrxValue    996 non-null    float64
 6   TotalTrxValue  996 non-null    float64
 7   Cash           996 non-null    int64  
 8   CreditCard     996 non-null    int64  
 9   Cheque         996 non-null    int64  
 10  SinceLastTrx   996 non-null    float64
 11  Churn          996 non-null    string 
dtypes: float64(5), int64(4), string(3)
memory usage: 101.2 KB
None


Finally, we save the data into a new and final dataset file using the following example Python code, naming it CustomerChurn.csv in a CVS format.

In [14]:
# save the data into a cvs file
df_merge.to_csv('data/CustomerChurn.csv')

Observe the file directory, the `CustomerChurn.csv` should be available for download and view. 

`We will use the CustomerChurn.csv dataset for future analysis and subsequent phases of the case study.`